In [52]:
from selenium import webdriver
import pandas as pd
import time

In [8]:
# from webdriver_manager.chrome import ChromeDriverManager
# driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
There is no [win32] chromedriver for browser 92.0.4515 in cache
Get LATEST driver version for 92.0.4515
Trying to download new driver from https://chromedriver.storage.googleapis.com/92.0.4515.107/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\Admin\.wdm\drivers\chromedriver\win32\92.0.4515.107]


In [54]:
data_from_excel = pd.read_excel("Chiến dịch Influencers.xlsx", sheet_name="Thống kê số views theo Link vid", index_col=None)

data = data_from_excel

In [55]:
data = data_from_excel

In [56]:
data.head()

,STT,Mã INF,Kênh hợp tác,Link Video
0,1,INFHUYEN641,Facebook,https://www.facebook.com/nguyen.huyenthu.716/v...
1,2,INFHUYEN641,Facebook,https://www.facebook.com/100008065904636/posts...
2,3,INFNHI376,TikTok,https://www.tiktok.com/@nhyyng/video/698771396...
3,4,INFGIANG354,TikTok,https://www.tiktok.com/@giangdeyy0/video/69862...
4,5,INFGIANG354,TikTok,https://www.tiktok.com/@giangdeyy0/video/69872...


In [57]:
Tiktok_data = data[data['Link Video'].str.startswith('https://www.tiktok.com/')]

In [58]:
# get rid of unwanted new line and spacing
Tiktok_data['Link Video'] = Tiktok_data['Link Video'].str.replace('\n', '', regex=True)
Tiktok_data['Link Video'] = Tiktok_data['Link Video'].str.replace(' ', '', regex=True)

# get rid of '?...'
Tiktok_data['Link Video'] = Tiktok_data['Link Video'].str.split("?", expand=True)[0]

# Some links are from tiktoke main page
Tiktok_data = Tiktok_data[Tiktok_data['Link Video'].str.contains("/video/")]

<ipython-input-58-642b7a96ae3e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Tiktok_data['Link Video'] = Tiktok_data['Link Video'].str.replace('\n', '', regex=True)
<ipython-input-58-642b7a96ae3e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Tiktok_data['Link Video'] = Tiktok_data['Link Video'].str.replace(' ', '', regex=True)
<ipython-input-58-642b7a96ae3e>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 

In [59]:
Tiktok_data['Link Video'].values

array(['https://www.tiktok.com/@nhyyng/video/6987713962817228059',
       'https://www.tiktok.com/@giangdeyy0/video/6986251851104554266',
       'https://www.tiktok.com/@giangdeyy0/video/6987221320555711770',
       'https://www.tiktok.com/@giangdeyy0/video/6987288042763504922',
       'https://www.tiktok.com/@giangdeyy0/video/6987566550714797338',
       'https://www.tiktok.com/@thuyngann_16/video/6988304363961765146',
       'https://www.tiktok.com/@shoppingfamily_/video/6985609926110432538',
       'https://www.tiktok.com/@thianguyen2001/video/6984759162777439514',
       'https://www.tiktok.com/@thianguyen2001/video/6985927736422370586',
       'https://www.tiktok.com/@thianguyen2001/video/6987770920102300954',
       'https://www.tiktok.com/@qh.bland/video/6987723855364115739',
       'https://www.tiktok.com/@soojin51296/video/6988474391059369243',
       'https://www.tiktok.com/@cu_bo32/video/6984019010870463771',
       'https://www.tiktok.com/@cu_bo32/video/6984622779647708442'

In [68]:
list1 = Tiktok_data['Link Video'].values[:4]
list2 = Tiktok_data['Link Video'].values[40:80]
list3 = Tiktok_data['Link Video'].values[80:]

In [69]:
list1

array(['https://www.tiktok.com/@nhyyng/video/6987713962817228059',
       'https://www.tiktok.com/@giangdeyy0/video/6986251851104554266',
       'https://www.tiktok.com/@giangdeyy0/video/6987221320555711770',
       'https://www.tiktok.com/@giangdeyy0/video/6987288042763504922'],
      dtype=object)

In [47]:
# Easily bypass notification

from selenium.webdriver.common.by import By

# Bypass notification

# chrome_options = webdriver.ChromeOptions()
# prefs = {"profile.default_content_setting_values.notifications" : 2}
# chrome_options.add_experimental_option("prefs",prefs)

driver = webdriver.Chrome() #chrome_options=chrome_options)

# Access google chrome and open insta login
driver.get("https://www.tiktok.com/login/")

In [72]:
my_list = []
last_url_check = ""

for link in list1:
    
    base_url = link[:-26]
    if (base_url == last_url_check):
        videos = driver.find_elements(By.XPATH, '//div[@class="jsx-2261688415 _ratio_wrapper"]/a') #.get_attribute("href")

        for video in videos:
            video_url = video.get_attribute("href")

            if (video_url == link):
                view = video.find_element_by_xpath('//strong[@class="jsx-3239500515 video-count"]').text

                item = {
                            "view": view,
                            "link": link
                }

                my_list.append(item)
    else:
    
        driver.get(base_url)
        
        last_url_check = base_url
        
        SCROLL_PAUSE_TIME = 2

        # Get scroll height
        last_height = driver.execute_script("return document.body.scrollHeight")

        while True:

            # Scroll down to bottom
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Wait to load page
            time.sleep(SCROLL_PAUSE_TIME)

            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

        videos = driver.find_elements(By.XPATH, '//div[@class="jsx-2261688415 _ratio_wrapper"]/a') #.get_attribute("href")

        for video in videos:
            video_url = video.get_attribute("href")

            if (video_url == link):
                view = video.find_element_by_xpath('//strong[@class="jsx-3239500515 video-count"]').text

                item = {
                            "view": view,
                            "link": link
                }

                my_list.append(item)
        